# Algo-trading: data preparation


> Author: <b>Nicola Bini</b>
>
> Date:   <b>06/17/2021</b>
>
>
> <b>Team 2:</b>
> <i>
> <br>Nicola Bini
> <br>Felipe Domingues
> <br>Tri Dung Dinh
> <br>Manuel Echazarra
> <br></i>

# Introduction



<b>Feature engineer variables and prepare data for the LSTM models.</b>

This is a part of our's team final submission for the course "Algorithmic trading with Python" taught by Professor Micheal Rolleigh at Hult University


## Input
This script takes as input data previously downloaded from yahoo finance!, for each stock we have its day-by-day closing price and its RSI.

We apply the same feature engineering to 4 sector stock indexes: 


## Output
The resulting dataframe contains the following columns for each stock:


<table>
  <tr>
    <th><b>Column</b></th>
    <th><b>Column description</b></th>
  </tr>
    
  <tr>
    <th>
        Date
    </th>
    <td>
        Date the other columns refer to
    </td>
  </tr>
  
  <tr>
    <th>
        Price
    </th>
    <td>
        Closing price of that day
    </td>
  </tr>
    
  <tr>
    <th>
        Price_%
    </th>
    <td>
        Percentage change from the previous day 
    </td>
  </tr>
    
  <tr>
    <th>
        RSI
    </th>
    <td>
        RSI indicator for that stock on a given day
    </td>
  </tr>
    
  <tr>
    <th>
        SMA
    </th>
    <td>
        Ratio between the short moving average and the long moving average (sma_short / sma_long)
    </td>
  </tr>
    
  <tr>
    <th>
        Profit
     </th>
    <td>
        It's 1 if the price increased from the previous day,<br> it's 0 if the price decreased or stayed the same from the previous day
    </td>
  </tr>
    
</table>


In [1]:
### Import libraries ###
import bt
import pandas      as pd
import pandas      as pd
import datetime
import numpy       as np
import keras

In [2]:
# Read data
data = pd.read_csv("data.csv")
rsi  = pd.read_csv("rsi.csv")

data = data.set_index("Date")
rsi = rsi.set_index("Date")

In [3]:
# define the length of the short and long moving averages
short = 50
long = 200

# Moving averages
sma_short = data.rolling(short).mean()
sma_long = data.rolling(long).mean()


In [4]:
data_index = data.index
data = data.reset_index(drop=True)

In [5]:
# Create a column for profit
df_profits = data.copy()
for col in data.columns:
    for i, row in data.iterrows():
        if i < len(data[col])-1:
            df_profits.loc[i, "profit_" + col] = data.loc[i, col] / data.loc[i+1, col] 
    
# Drop old columns and set date as index
df_profits["date"] = data_index
df_profits = df_profits.set_index("date")
df_profits.drop(data.columns, axis=1, inplace=True)

In [6]:
# Transform values into labels: profit/non-profit

for col in df_profits:
    for i, row in df_profits.iterrows():
        if df_profits.loc[i, col] > 1:
            df_profits.loc[i, col] = 1
        elif df_profits.loc[i, col] <= 1:
            df_profits.loc[i, col] = 0

In [7]:
df_profits.describe()

,profit_DNN,profit_HGEN,profit_ICLN,profit_OCGN,profit_VGT,profit_VUZI,profit_XLV,profit_^NBI
count,1639.000000,1639.000000,1639.000000,1639.000000,1639.000000,1639.000000,1639.000000,1639.000000
mean,0.390482,0.452105,0.455766,0.528371,0.425869,0.492373,0.458206,0.465528
std,0.488007,0.497853,0.498191,0.499347,0.494625,0.500094,0.498402,0.498962
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
sma_normalized = sma_short.copy()

# Normalize SMA
for col in sma_short.columns:
    for i, row in sma_short.iterrows():
        sma_normalized.loc[i, col] = sma_short.loc[i, col] / sma_long.loc[i,col]
        
    sma_normalized["sma_" + col] = sma_normalized[col]
    sma_normalized.drop(col, axis=1, inplace=True)
        
# Normalize price

for col in data.columns:
    percentage_price = "price%_" + col
    data[percentage_price] = 0
    for i, row in data.iterrows():
        if i != 0:
            prev_i = i - 1
            if data.loc[prev_i, col] != 0:
                data.loc[i, percentage_price] = (data.loc[i, col] - data.loc[prev_i, col]) /  data.loc[prev_i, col]
            else:
                data.loc[i, percentage_price] = 1
            
data["date"] = data_index
data = data.set_index("date")

In [9]:
# Check normalized sma
sma_normalized.describe()

,sma_DNN,sma_HGEN,sma_ICLN,sma_OCGN,sma_VGT,sma_VUZI,sma_XLV,sma_^NBI
count,1441.000000,1441.000000,1441.000000,1441.000000,1441.000000,1441.000000,1441.000000,1441.000000
mean,1.011255,1.008159,1.043781,0.869971,1.063155,1.057540,1.023215,1.013430
std,0.205478,0.518908,0.131658,0.642581,0.051581,0.345771,0.033830,0.064070
min,0.619741,0.243615,0.875828,0.063517,0.936117,0.556436,0.940654,0.811207
25%,0.900042,0.667836,0.961906,0.561413,1.030844,0.869102,0.998349,0.979384
50%,0.981710,0.928250,1.026100,0.795189,1.073722,0.967275,1.028140,1.022407
75%,1.101563,1.140447,1.077791,1.077065,1.091721,1.160074,1.051611,1.059666
max,1.732039,2.745353,1.497245,3.385863,1.164801,2.263693,1.083949,1.127636


In [10]:
final_data_1 = pd.concat([data, rsi, sma_normalized, df_profits],axis=1)

In [11]:
# The moving average and the profit columns generated null values, drop them.
# (moving average starts after 200 values and we don't know the price of tomorrow)
final_data = pd.concat([data, rsi, sma_normalized, df_profits],axis=1)
final_data = final_data.iloc[199:,:]
final_data = final_data.iloc[:-1,:]
final_data.isnull().sum()

DNN            0
HGEN           0
ICLN           0
OCGN           0
VGT            0
VUZI           0
XLV            0
^NBI           0
price%_DNN     0
price%_HGEN    0
price%_ICLN    0
price%_OCGN    0
price%_VGT     0
price%_VUZI    0
price%_XLV     0
price%_^NBI    0
RSI_DNN        0
RSI_HGEN       0
RSI_ICLN       0
RSI_OCGN       0
RSI_VGT        0
RSI_VUZI       0
RSI_XLV        0
RSI_^NBI       0
sma_DNN        0
sma_HGEN       0
sma_ICLN       0
sma_OCGN       0
sma_VGT        0
sma_VUZI       0
sma_XLV        0
sma_^NBI       0
profit_DNN     0
profit_HGEN    0
profit_ICLN    0
profit_OCGN    0
profit_VGT     0
profit_VUZI    0
profit_XLV     0
profit_^NBI    0
dtype: int64

In [12]:
final_data_1.columns

Index(['DNN', 'HGEN', 'ICLN', 'OCGN', 'VGT', 'VUZI', 'XLV', '^NBI',
       'price%_DNN', 'price%_HGEN', 'price%_ICLN', 'price%_OCGN', 'price%_VGT',
       'price%_VUZI', 'price%_XLV', 'price%_^NBI', 'RSI_DNN', 'RSI_HGEN',
       'RSI_ICLN', 'RSI_OCGN', 'RSI_VGT', 'RSI_VUZI', 'RSI_XLV', 'RSI_^NBI',
       'sma_DNN', 'sma_HGEN', 'sma_ICLN', 'sma_OCGN', 'sma_VGT', 'sma_VUZI',
       'sma_XLV', 'sma_^NBI', 'profit_DNN', 'profit_HGEN', 'profit_ICLN',
       'profit_OCGN', 'profit_VGT', 'profit_VUZI', 'profit_XLV',
       'profit_^NBI'],
      dtype='object')

In [13]:
# Save data
final_data.to_csv("final_data.csv")